In [ ]:
!pip -q install langchain openai tiktoken chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 759.0/759.0 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 34.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.4/46.4 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 922.4/922.4 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 2.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 364.5 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip show langchain

Name: langchain
Version: 0.0.161
Summary: Building applications with LLMs through composability
Home-page: https://www.github.com/hwchase17/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, async-timeout, dataclasses-json, numexpr, numpy, openapi-schema-pydantic, pydantic, PyYAML, requests, SQLAlchemy, tenacity, tqdm
Required-by: 


In [ ]:
!wget -q https://www.dropbox.com/s/vs6ocyvpzzncvwh/new_articles.zip
!unzip -q new_articles.zip -d new_articles

# LangChain multi-doc retriever with ChromaDB

***New Points***
- Multiple Files
- ChromaDB
- Source info
- gpt-3.5-turbo API

## Setting up LangChain


In [1]:
import os

OPEN_API_KEY = os.getenv("OPENAI_API_KEY", None)

In [9]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader
import time


## Load multiple and process documents

In [3]:
# Load and process the text files
# loader = TextLoader('single_text_file.txt')
loader = DirectoryLoader('./data/', glob="./*.pdf", loader_cls=PyPDFLoader)

documents = loader.load()

In [208]:
#splitting the text into
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

In [209]:
len(texts)

62

In [210]:
texts[3]

Document(page_content='appraisals of social threat: paranoia concerns imminent and ongoing physical, psychological or social harms by \n others7, whereas social anxiety reflects worry about rejection, embarrassment and  scrutiny9. Among individuals \nwith first-episode psychosis, those with comorbid social anxiety disorder reported more persecutory threats than \nthose without  comorbidity3. Across non-patient and community samples, correlations between subclinical levels \nof social anxiety and paranoia are consistently found to be moderate to  strong10–13.\nThe co-occurrence of social anxiety and paranoia raises questions about how the two symptoms may influ-\nence each  other3,14. On the one hand, it has been proposed that paranoia develops against the backdrop of \nanxiety and related worry  processes15–17. The cognitive model of paranoia posits that social evaluative concerns, OPEN\n1Department of Psychology, The Chinese University of Hong Kong, Hong Kong Special Administrative Re

## create the DB

In [211]:
%%time
# Embed and store the texts
# Supplying a persist_directory will store the embeddings on disk
persist_directory = 'db'

## here we are using OpenAI embeddings but in future we will swap out to local embeddings
embedding = OpenAIEmbeddings()

vectordb = Chroma.from_documents(documents=texts,
                                 embedding=embedding,
                                 persist_directory=persist_directory)



Using embedded DuckDB with persistence: data will be stored in: db


CPU times: user 201 ms, sys: 45.7 ms, total: 246 ms
Wall time: 4.27 s


In [212]:
# persiste the db to disk
vectordb.persist()
vectordb = None

In [213]:
# Now we can load the persisted database from disk, and use it as normal.
vectordb = Chroma(persist_directory=persist_directory,
                  embedding_function=embedding)

Using embedded DuckDB with persistence: data will be stored in: db


## Make a retriever

In [214]:
retriever = vectordb.as_retriever()

In [215]:
docs = retriever.get_relevant_documents("What is paranoia?")

In [216]:
len(docs)

2

In [217]:
retriever = vectordb.as_retriever(search_kwargs={"k": 2})

In [218]:
retriever.search_type

'similarity'

In [219]:
retriever.search_kwargs

{'k': 2}

## Make a chain

In [220]:
# create the chain to answer questions
qa_chain = RetrievalQA.from_chain_type(llm=OpenAI(),
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True)

In [221]:
## Cite sources
def process_llm_response(llm_response):
    print(llm_response['result'])
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [222]:
# full example
query = "What is paranoia?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 Paranoia is a type of thinking which involves suspiciousness, distrust, and fear of others.


Sources:
data/s41598-023-47912-0.pdf
data/s41598-023-47912-0.pdf


In [223]:
# break it down
query = "How many young adults (or people) took part in this?"
llm_response = qa_chain(query)
# process_llm_response(llm_response)
llm_response

{'query': 'How many young adults (or people) took part in this?',
 'result': " This information is not provided in the context, so I don't know.",
 'source_documents': [Document(page_content='(2021).\n 43. Jefferies, P . & Ungar, M. Social anxiety in young people: A prevalence study in seven countries. PLOS ONE  15, e0239133. https://  \ndoi. org/ 10. 1371/  journ  al. pone.  02391  33 (2020).\n 44. Freeman, D. et al. Concomitants of paranoia in the general population. Psychol. Med. 41, 923–936. https:// doi. org/ 10. 1017/ S0033  \n29171 00015  46 (2011).\n 45. Lim, M. H. et al. A pilot digital intervention targeting loneliness in young people with psychosis. Soc. Psychiat. Psychiat. Epidemiol.  \n55, 877–889. https://  doi. org/ 10. 1007/  s00127-  019-  01681-2  (2020).\n 46. Lim, M. H., Penn, D. L., Thomas, N. & Gleeson, J. F. M. Is loneliness a feasible treatment target in psychosis?. Soc. Psychiat. Psychiat.', metadata={'source': 'data/s41598-023-47912-0.pdf', 'page': 8}),
  Docu

In [224]:
query = "How do they measure Momentary social anxiety?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 Momentary social anxiety is measured with the three items suggested in Kashdan and Steger (e.g., ‘I worried that I would say or do something wrong right now’). In the current study, the within- and between-person reliabilities were 0.84 and 0.99 respectively.


Sources:
data/s41598-023-47912-0.pdf
data/s41598-023-47912-0.pdf


In [194]:
query = "What is their data collection method?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 Face-to-face data collection.


Sources:
data/s41598-023-47912-0.pdf


In [195]:
query = "What is ESM?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 I don't know.


Sources:
data/s41598-023-47912-0.pdf
data/s41598-023-47912-0.pdf


In [196]:
query = "What is the result of this study?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 This article does not report the results of a study; it provides information about the acknowledgements, author contributions, funding, competing interests, and additional information related to the study.


Sources:
data/s41598-023-47912-0.pdf
data/s41598-023-47912-0.pdf


In [197]:
query = "What is the limitations of the current study?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 The limitations of the current study include that the results may be specific to the current sampling frequency of ESM assessment, that data collection was conducted during the COVID-19 pandemic, that a majority of the sample were undergraduate students, and that there may be other unmeasured mechanisms involved in the dynamics between social anxiety and paranoia.


Sources:
data/s41598-023-47912-0.pdf
data/s41598-023-47912-0.pdf


In [198]:
query = "What is the hypothesis of the study?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 I don't know.


Sources:
data/s41598-023-47912-0.pdf


In [199]:
query = "What is the final sample size of the study?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 134 participants.


Sources:
data/s41598-023-47912-0.pdf
data/s41598-023-47912-0.pdf


In [200]:
query = "Where did the study take place?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 The study took place at The Chinese University of Hong Kong.


Sources:
data/s41598-023-47912-0.pdf
data/s41598-023-47912-0.pdf


In [233]:
query = "Who write this report?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 S.H.S. and E.B., with A.K.C.C. contributing to the statistical analysis and helping to write the first draft of the manuscript. All authors contributed to and approved the final manuscript.


Sources:
data/s41598-023-47912-0.pdf


In [234]:
query = "Who are the authors of this report?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 S.H.S. and E.B., A.K.C.C.


Sources:
data/s41598-023-47912-0.pdf


In [238]:
query = "How's the weather today?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 I don't know.


Sources:
data/s41598-023-47912-0.pdf


In [201]:
qa_chain.retriever.search_type , qa_chain.retriever.vectorstore

('similarity', <langchain.vectorstores.chroma.Chroma at 0x168cd46a0>)

In [202]:
print(qa_chain.combine_documents_chain.llm_chain.prompt.template)

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Helpful Answer:


## Agent implementation

In [225]:
from langchain.agents import initialize_agent, Tool

In [228]:
# the zero-shot-react-description agent will use the "description" string to select tools
tools = [
    Tool(
        name = "idk",
        func=qa_chain.run,
        description="""useful for when a user is interested in various Neo4j information, 
                       use-cases, or applications. A user is not asking for any debugging, but is only
                       interested in general advice for integrating and using Neo4j.
                       Input should be a fully formed question."""
    )
]

In [236]:
agent = initialize_agent(tools, OpenAI(), agent="zero-shot-react-description", verbose=True)

In [240]:
agent.run("""How's the weather today?""")



> Entering new AgentExecutor chain...
 This isn't a question about Neo4j
Action: idk
Action Input: weather

ValueError: `run` not supported when there is not exactly one output key. Got ['result', 'source_documents'].